# Initial setting

In [49]:
import random
# Total number of teams, around 2-2.5k according to discord. To make the digs attractive, i chose 2k
total_teams = 2000

# Strategy distribution: 1 dig, 2 digs, 3 digs
strategy_distribution = [0.3, 0.3, 0.4]

# Some might have multiple chests, so let's say the expected outcome of one chest is 7500 + 0.2 * 7500 = 9000
expected_chest_profit = 9000

# cost for each dig
dig_costs = [0, 25000, 75000]

# Define the tiles with their corresponding multipliers and number of hunters
tiles = {
    "G26": (24, 2), "G27": (70, 4), "G28": (41, 3), "G29": (21, 2), "G30": (60, 4),
    "H26": (47, 3), "H27": (82, 5), "H28": (87, 5), "H29": (80, 5), "H30": (35, 3), 
    "I26": (73, 4), "I27": (89, 5), "I28": (100, 8), "I29": (90, 7), "I30": (17, 2),
    "J26": (77, 5), "J27": (83, 5), "J28": (85, 5), "J29": (79, 5), "J30": (55, 4), 
    "K26": (12, 2), "K27": (27, 3), "K28": (52, 4), "K29": (15, 2),  "K30": (30, 3)
}

# Function to calculate expected profit for a tile
def expected_profit(tile_info):
    multiplier, hunters = tile_info
    profit = (expected_chest_profit * multiplier) / hunters
    return profit



## If all the teams are randomly distributed, what will happen for the profit of each spot?

In [58]:
# Initialize the count of hunters for each tile
tile_hunters = {tile: info[1] for tile, info in tiles.items()}

# Randomly distribute all teams across the tiles with one dig each
for _ in range(total_teams):
    chosen_tile = random.choice(list(tiles.keys()))
    tile_hunters[chosen_tile] += 1

# Calculate updated profits for each tile
updated_profits = {}
for tile, (multiplier, _) in tiles.items():
    updated_hunters = tile_hunters[tile]
    updated_profit = expected_profit((multiplier, updated_hunters))
    updated_profits[tile] = updated_profit

# Sort tiles by updated profit in descending order
sorted_tiles_by_updated_profit = sorted(updated_profits.items(), key=lambda item: item[1], reverse=True)

# Display expected profits after all teams are distributed, in descending order
print("Expected profit for each tile after random distribution (sorted):")
for tile, profit in sorted_tiles_by_updated_profit:
    print(f"{tile}: {profit:.2f} seashells")


Expected profit for each tile after random distribution (sorted):
I27: 11608.70 seashells
J27: 11492.31 seashells
I29: 10125.00 seashells
J28: 10065.79 seashells
I28: 9782.61 seashells
J26: 8555.56 seashells
H27: 8386.36 seashells
H29: 8181.82 seashells
H28: 8156.25 seashells
J29: 7988.76 seashells
G27: 7411.76 seashells
I26: 7141.30 seashells
J30: 6111.11 seashells
G30: 5806.45 seashells
K28: 5379.31 seashells
H26: 4862.07 seashells
H30: 4256.76 seashells
G28: 4100.00 seashells
K30: 3461.54 seashells
K27: 2963.41 seashells
G26: 2805.19 seashells
G29: 2054.35 seashells
K29: 1708.86 seashells
I30: 1471.15 seashells
K26: 1440.00 seashells


### We can draw a conclusion, if every team is randomly distributed, we should never conduct a second dig

### What if they are not randomly distributed?

Assumptions:
- 30% of the teams are stupid, they conduct 1 dig, and they only focus on the top 10 profitable spots and least 10 profitable spots based on treasure map
- 40% of the teams know sth, but they don't know everything, so they would randomly distributed across the treasure maps. they would conduct one dig.
- 30% of the teams are supermen, they know everything, not only the treasure map. So they would only choose the spot that are the most profitable to them, they only conduct one dig.

In [55]:
team_distribution = {
    'stupid': int(total_teams * 0.3),
    'know_something': int(total_teams * 0.4),
    'supermen': int(total_teams * 0.3),
}

# Calculate initial expected profit for sorting
initial_expected_profit = {tile: (expected_chest_profit * multiplier) / hunters for tile, (multiplier, hunters) in tiles.items()}
sorted_tiles_by_profit = sorted(initial_expected_profit.items(), key=lambda item: item[1], reverse=True)

# Define the targets for each team type
top_10 = [tile for tile, _ in sorted_tiles_by_profit[:10]]
bottom_10 = [tile for tile, _ in sorted_tiles_by_profit[-10:]]
most_profitable = sorted_tiles_by_profit[0][0]

# Initialize the count of hunters for each tile based on team strategies
tile_hunters = {tile: info[1] for tile, info in tiles.items()}

# "Stupid" teams target the top 10 and bottom 10 profitable spots
for tile in top_10 + bottom_10:
    tile_hunters[tile] += team_distribution['stupid'] // 20  # 20 spots in total

# "Know something" teams are randomly distributed
for _ in range(team_distribution['know_something']):
    chosen_tile = random.choice(list(tiles.keys()))
    tile_hunters[chosen_tile] += 1

# "Supermen" teams each choose the most profitable slot available, considering other supermen's choices
for _ in range(team_distribution['supermen']):
    # Recalculate profitability based on the current state
    current_profitability = {tile: expected_profit((multiplier, tile_hunters[tile])) for tile, (multiplier, _) in tiles.items()}
    # Choose the most profitable slot for this "superman"
    best_slot_for_superman = max(current_profitability, key=current_profitability.get)
    # Update the hunters count for this chosen slot
    tile_hunters[best_slot_for_superman] += 1

# Recalculate and display expected profits after the distribution
updated_expected_profit = {tile: (expected_chest_profit * multiplier) / tile_hunters[tile] for tile, (multiplier, _) in tiles.items()}

sorted_expeditions = sorted(updated_expected_profit.items(), key=lambda item: item[1], reverse=True)

print("Expeditions sorted by expected profit after modeling:")
for tile, profit in sorted_expeditions:
    print(f"{tile}: {profit:.2f} seashells")

Expeditions sorted by expected profit after modeling:
G30: 7200.00 seashells
H29: 7200.00 seashells
I28: 7200.00 seashells
K28: 7200.00 seashells
H28: 7183.49 seashells
J27: 7182.69 seashells
J29: 7181.82 seashells
J30: 7173.91 seashells
I29: 7168.14 seashells
H27: 7165.05 seashells
G27: 7159.09 seashells
I27: 7151.79 seashells
J28: 7149.53 seashells
J26: 7144.33 seashells
I26: 7141.30 seashells
G28: 7096.15 seashells
H26: 6714.29 seashells
K30: 4821.43 seashells
H30: 4038.46 seashells
K27: 3919.35 seashells
G29: 3098.36 seashells
G26: 2842.11 seashells
I30: 2467.74 seashells
K29: 2076.92 seashells
K26: 1611.94 seashells


# Let's brutal force the distribution

In [57]:
know_something_ratio = 0.4  # Fixed ratio for "know something" teams

for stupid_ratio in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]:
    supermen_ratio = 1 - (stupid_ratio + know_something_ratio) 

    # Calculate initial expected profit for sorting
    initial_expected_profit = {tile: expected_profit((multiplier, hunters)) for tile, (multiplier, hunters) in tiles.items()}
    sorted_tiles_by_profit = sorted(initial_expected_profit.items(), key=lambda item: item[1], reverse=True)

    # Define the targets for each team type
    top_10 = [tile for tile, _ in sorted_tiles_by_profit[:10]]
    bottom_10 = [tile for tile, _ in sorted_tiles_by_profit[-10:]]
    
    # Initialize the count of hunters for each tile based on team strategies
    tile_hunters = {tile: info[1] for tile, info in tiles.items()}

    # "Stupid" teams target the top 10 and bottom 10 profitable spots
    for tile in top_10 + bottom_10:
        tile_hunters[tile] += int((total_teams * stupid_ratio) // 20)  # 20 spots in total

    # "Know something" teams are randomly distributed
    for _ in range(int(total_teams * know_something_ratio)):
        chosen_tile = random.choice(list(tiles.keys()))
        tile_hunters[chosen_tile] += 1

    # "Supermen" teams each choose the most profitable slot available, considering other supermen's choices
    for _ in range(int(total_teams * supermen_ratio)):
        current_profitability = {tile: expected_profit((multiplier, tile_hunters[tile])) for tile, (multiplier, _) in tiles.items()}
        best_slot_for_superman = max(current_profitability, key=current_profitability.get)
        tile_hunters[best_slot_for_superman] += 1

    # Recalculate and display expected profits after the distribution
    updated_expected_profit = {tile: expected_profit((multiplier, tile_hunters[tile])) for tile, (multiplier, _) in tiles.items()}
    sorted_expeditions = sorted(updated_expected_profit.items(), key=lambda item: item[1], reverse=True)

    print(f"\nDistribution with {stupid_ratio*100}% 'Stupid' Teams:")
    for tile, profit in sorted_expeditions[:3]:  # Display top 5 for brevity
        print(f"{tile}: {profit:.2f} seashells")



Distribution with 10.0% 'Stupid' Teams:
J29: 6463.64 seashells
I27: 6459.68 seashells
I26: 6441.18 seashells

Distribution with 20.0% 'Stupid' Teams:
G28: 6833.33 seashells
H27: 6833.33 seashells
J28: 6830.36 seashells

Distribution with 30.0% 'Stupid' Teams:
I29: 7105.26 seashells
G28: 7096.15 seashells
H27: 7096.15 seashells

Distribution with 40.0% 'Stupid' Teams:
I26: 7551.72 seashells
K28: 7548.39 seashells
J27: 7545.45 seashells

Distribution with 50.0% 'Stupid' Teams:
I29: 8265.31 seashells
I27: 8257.73 seashells
I28: 8256.88 seashells

Distribution with 60.0% 'Stupid' Teams:
J26: 21000.00 seashells
K28: 13371.43 seashells
J30: 12073.17 seashells
